In [85]:
from transformers import XLNetTokenizer, XLNetLMHeadModel,XLNetConfig
import sentencepiece as spm
from chunked_text_dataloader import ChunkedTextDataset
import torch
import pandas as pd
import numpy as np
import os

In [67]:
os.getcwd()

'D:\\PROJECTS\\python\\astro'

https://nonint.com/2020/03/27/fine-tuning-xlnet-for-generation-tasks/

In [25]:
data=pd.read_csv('data.csv')

In [72]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-large-cased", force_download=False, cache_dir="tokenizers")

In [64]:
output = []
for i in range(len(data)):
    target = data["text"][i]
    target_enc = tokenizer.encode(target, add_special_tokens=False, max_length=None, pad_to_max_length=False)
    text = data["sign"][i]+' '+ data["date"][i]
    text_enc = tokenizer.encode(text, add_special_tokens=False, max_length=None, pad_to_max_length=False)
    output.append({
        "target": torch.tensor(target_enc, dtype=torch.long),
        "text": torch.tensor(text_enc, dtype=torch.long)
    })

torch.save(output, "out.pt")

In [81]:
max_seq_len=5
predict_len=190
batch_size=5

In [83]:
# chunked_model_config is a dictionary initialized from command line arguments. Most fields
# are self-explanatory or can be inferred from the ChunkedTextDataset docs.
# Get the datasets
input_folder=os.getcwd()

train_set = ChunkedTextDataset(
    "out.pt",
    tokenizer,
    max_seq_len,
    predict_len,
    pad_left=True,
)
val_set = ChunkedTextDataset(
    "out.pt",
    tokenizer,
    max_seq_len,
    predict_len,
    pad_left=True,
)
train_loader = train_set.get_dataloader(batch_size, num_workers=0)


In [88]:
config = XLNetConfig.from_pretrained('xlnet-large-cased', cache_dir="config")
model = XLNetLMHeadModel.from_pretrained( config=config, from_tf=True)

#config.mem_len = chunked_model_config["mem_len"]
# config.num_labels = 1 # Un-comment this if you want 
#model = transformers.XLNetLMHeadModel.from_pretrained(
#        chunked_model_config["model_name"], 

TypeError: from_pretrained() missing 1 required positional argument: 'pretrained_model_name_or_path'

In [ ]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [
            p
            for n, p in model.named_parameters()
            if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0,
    },
    {
        "params": [
            p
            for n, p in model.named_parameters()
            if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]
optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=start_lr, eps=1e-8)
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=epochs * len(train_set) / aggregate_batch_size,
)